

# Classic Elo

Classic elo algorithm is great for establishing a baseline for comparison of other methods. Due to its age and simplicity, it will almost certainly not beat any liquid markets. However, it shines when you have a lot of historical data and there are jumps in class. I.e., in college football, classic Elo will properly rate a strong group of five team even after they've played 10 easy opponents. In other words, it can store information about historically good conferences for a long time despite no interplay between conferences. Another example is in soccer when a newly promoted side enters a better division. Despite the promoted team having won 60%+ of their games the prior year, Elo will not project a gigantic winning percentage going forward against tougher competition. Elo also has the advantage that you do not need to know underlying distributions, only win/loss. 

### Advantages

- Simplicity
- Computational speed
- Universality
- Good at handling cross-league, cross-conference play

### Disadvantages

- No uncertainty. This causes problems when there are new players that you have no idea how good they are, or a player coming off extended absence
- Cannot account for score difference. A win by 50 is the same as a win by 5.
- Constant K factor. If players try harder in some games than others, it is not intrinsically capable of handling that
- As with any rating system, if players are allowed to selectively choose opponents, it can ruin the fidelity of the ratings
- Cannot account for teammate ratings



In [40]:


import os

import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy.optimize import minimize
from pandas.api.types import is_datetime64_any_dtype as is_datetime


### customize to your own
DATA_PATH = 'D://Medium/'



In [2]:


def load_data():
    return pd.read_csv(os.path.join(DATA_PATH, 'ncaam_sample_data.csv'))

def process(data):
    
    data['team_reb'] = data['team_or'].copy()+data['team_dr'].copy()
    data['opp_reb'] = data['opp_or'].copy()+data['opp_dr'].copy()
    data = data.copy()[['date','season','team_name','opp_name','is_home','team_score','opp_score', 'team_reb','opp_reb']]
    data['date'] = pd.to_datetime(data['date'])
    data['score_diff'] = data['team_score'].copy()-data['opp_score'].copy()
    data['reb_diff'] = data['team_reb'].copy()-data['opp_reb'].copy()
    
    ## long format preferred
    data =data.melt(
        id_vars=['date','season','team_name','opp_name','is_home'], 
        value_vars=['score_diff','reb_diff'], 
        var_name='stat', 
        value_name='difference'
    )
    
    data['result'] = np.where(data['difference']>0, 1, 0)
    data['result'] = np.where(data['difference']==0, 0.5, data['result'].copy())
    
    data = data.sort_values(by=['date','team_name','stat']).reset_index(drop=True)
    
    return data

m_data = load_data()
m_data = process(m_data)


In [10]:
m_data.is_home.value_counts()

-1    183354
 1    183354
 0     46412
Name: is_home, dtype: int64

In [35]:

class EloClassic():
    
    """
    
    Implements classic Elo algorithm for multiple stats
    
    """
    
    
    def __init__(self, 
                 data, 
                 k,
                 hfa,
                 protag_id='team_name', 
                 antag_id='opp_name',
                 result_col='result',
                 priors = None
                ):
        
        self.data = data
        self.k=k
        assert(type(k)==dict), "k must be a dictionary whose keys are the stat names and values are the k factor for that stat"
        self.protag_id = protag_id
        self.antag_id = antag_id
        assert('stat' in list(self.data)), 'No stat column'
        self.stats = list(self.data.stat.unique())
        self.hfa = hfa
        assert(type(hfa)==dict), "hfa must be a dictionary whose keys are the stat names and values are the hfa for that stat.\n Use zero if home field advantage doesn't apply"
        self.result_col = result_col
        self.priors = priors
        
        results = list(self.data[self.result_col].unique())
        assert(all([(np.isclose(r,0)|(np.isclose(r,1)|(np.isclose(r,0.5)))) for r in results])), "Results must be zero (for loss) or one (for win) or 0.5 (for tie)"
        
        if 'is_home' not in list(self.data):
            self.data['is_home']=0
            for stat in self.stats:
                self.hfa[stat] = 0
        locs = (self.data['is_home'].unique())
        assert(all([(np.isclose(l,0)|(np.isclose(l,1)|(np.isclose(l,-1)))) for l in locs])), "is_home col needs either 1 for home, -1 for away, or 0 for neutral"

        col_names = list(self.data)
        col_names = [cn.lower().strip() for cn in col_names]
        assert((('date' in col_names)|('rating_period' in col_names))), "Need either a date column or a rating period column"
        self.data.columns=col_names
        
        if 'date' in col_names:
            date_dtype = self.data.date.dtype
            assert(is_datetime(date_dtype)), "Date column must be of type datetime"
        else:
            date_dtype_check = True
        
        if (('date' in col_names) & ('rating_period' not in col_names)):
            self.data['rating_period'] = self.data.date.copy().rank(method='dense')

        self.protag_ids = set(self.data[self.protag_id].unique())
        self.antag_ids = set(self.data[self.antag_id].unique())
        
        assert(len(self.protag_ids.symmetric_difference(self.antag_ids))==0), "In SPR format, need a row for each team in dataframe (two rows per game)"
        
        self.num_stats = len(self.stats)
        self.num_protags = len(self.protag_ids)
        self.num_games = len(self.data)//2
        
        self.protag2index = {}
        for i,protag_id in enumerate(self.protag_ids):
            self.protag2index[protag_id] = i
            
        self.stat2index = {}
        for j,stat in enumerate(self.stats):
            self.stat2index[stat] = j
            
        self.data['protag_idx'] = self.data[self.protag_id].map(self.protag2index)
        self.data['antag_idx'] = self.data[self.antag_id].map(self.protag2index)
        self.data['stat_idx'] = self.data['stat'].map(self.stat2index)
        self.data['hfa'] = self.data['stat'].map(self.hfa).copy()*self.data['is_home'].copy()
        assert(len(self.data.loc[self.data.hfa.isnull()])==0), f"{self.data.loc[self.data.hfa.is_null()].stat.unique()} do not have a home field advantage number"
        
        self.data['k'] = self.data['stat'].map(self.k).copy()
        assert(len(self.data.loc[self.data.k.isnull()])==0), f"{self.data.loc[self.data.k.is_null()].stat.unique()} do not have a k factor specified in the k factor dict"

        self.rating_matrix = np.ones((self.num_protags, self.num_stats))*1500
        
        return
    
    def opt_helper(self, params):
        
        
        
        _, grade = self.history(k=k, hfa=hfa)
        
        return grade
    
    def _rating_period_update(self, protag_ratings, antag_ratings, k, results):
        """
        performs classic Elo rating update calculation
        """
        
        probs = 1/(1+10**((antag_ratings-protag_ratings)/400))
        return k*(results - probs)
    
    def info(self):
        """
        returns meta info, usually used right after initialization
        """
        
        print(f"There are {self.num_stats} stats: {self.stats}")
        print(f"There are {self.num_protags:,} unique players/teams.")
        print(f"There are {self.num_games:,} games from {self.data.date.min()} to {self.data.date.max()}.")
        
        return
    
    
    def history(self, k=None, hfa=None, metric='brier'):
        """
        calculates entire pre-game (non-leaky) ratings using current parameters
        
        returns: those ratings
        """
        
        assert(metric in ['brier','log_loss']), 'please use an implemented metric (brier, log_loss)' 
        if k is None:
            k = self.k
        if hfa is None:
            hfa = self.hfa
        
        history = []
        quick_iterator = self.data.groupby(['rating_period'])
        for rp_index, rating_period in tqdm(quick_iterator, total=len(quick_iterator)):
            
            ## append pregame ratings to history
            pregame_protag_ratings = self.rating_matrix[rating_period.protag_idx.values, rating_period.stat_idx.values]
            pregame_antag_ratings = self.rating_matrix[rating_period.antag_idx.values, rating_period.stat_idx.values]

            to_append = rating_period[['date',self.protag_id,self.antag_id,'is_home','hfa','stat',self.result_col]].copy()
            to_append['pregame_rating'] = pregame_protag_ratings
            to_append['pregame_opp_rating'] = pregame_antag_ratings
            
            if self.hfa is not None:
                ## account for hfa
                pregame_protag_ratings = pregame_protag_ratings+(rating_period.is_home.values*rating_period.hfa.values)
            
            rating_adjustments = self._rating_period_update(pregame_protag_ratings, pregame_antag_ratings, rating_period.k.values, rating_period.result.values)
            
            if self.hfa is not None:
                ## reset ratings
                pregame_protag_ratings = pregame_protag_ratings-(rating_period.is_home.values*rating_period.hfa.values)
                
            ## apply update
            new_ratings = pregame_protag_ratings+rating_adjustments
            
            history.append(to_append)
            
            self.rating_matrix[rating_period.protag_idx.values, rating_period.stat_idx.values] = new_ratings

        self.history = pd.concat(history, axis=0).reset_index(drop=True)
        
        self.history['rtg_diff'] = self.history['pregame_opp_rating'].copy()-(self.history['pregame_rating'].copy()+(self.history['is_home'].copy()*self.history.hfa.values))
        self.history['probability'] = 1/(1+10**((self.history['rtg_diff'])/400))
        if metric == 'brier':
            ## allow some time for ratings to stabilize
            initial = int(0.15*len(self.history))
            score = ((self.history[initial:]['result'].copy()-self.history[initial:]['probability'].copy())**2).mean()
        self.history = self.history.drop(columns=['rtg_diff'])
        
        return self.history, score
    
    
    def optimize(self, tol=0.001, method='Nelder-Mead'):
        """
        optimizes k value and home field advantage
        """
        x0 = np.array(list(self.k.values())+list(self.hfa.values()))
        bounds = [(0, None) for i in range(x0)]
        res = minimize(grade_history, x0, method='Nelder-Mead', tol=1e-6)
        res.x
        
        return
    
    
    def update(self):
        
        return
    
    def predict(self):
        
        return
    
    


EC = EloClassic(m_data, k={'score_diff':15, 'reb_diff':10}, hfa={'score_diff':35, 'reb_diff':35})
EC.info()
    
    

There are 2 stats: ['reb_diff', 'score_diff']
There are 363 unique players/teams.
There are 206,560 games from 2002-11-14 00:00:00 to 2022-04-04 00:00:00.


In [36]:

history, score = EC.history()


100%|████████████████████████████████████████████████████████████████████████████| 2674/2674 [00:01<00:00, 1677.83it/s]


In [39]:


history.loc[history['is_home']==1]


,date,team_name,opp_name,is_home,hfa,stat,result,pregame_rating,pregame_opp_rating,probability
22,2002-11-15,Wisconsin,E Washington,1,35,reb_diff,1.0,1500.000000,1500.000000,0.550199
23,2002-11-15,Wisconsin,E Washington,1,35,score_diff,1.0,1500.000000,1500.000000,0.550199
26,2002-11-16,Colorado St,PFW,1,35,reb_diff,1.0,1500.000000,1500.000000,0.550199
27,2002-11-16,Colorado St,PFW,1,35,score_diff,1.0,1500.000000,1500.000000,0.550199
38,2002-11-16,Wisconsin,N Illinois,1,35,reb_diff,1.0,1504.498006,1495.000000,0.563690
...,...,...,...,...,...,...,...,...,...,...
412297,2022-03-09,Vermont,Binghamton,1,35,score_diff,1.0,1353.104852,904.426026,0.941820
412386,2022-03-10,Kansas,West Virginia,1,35,reb_diff,1.0,1535.988251,1447.060581,0.671149
412387,2022-03-10,Kansas,West Virginia,1,35,score_diff,1.0,1701.171001,1475.600866,0.817569
412824,2022-03-12,Vermont,UMBC,1,35,reb_diff,1.0,1398.221360,1201.471878,0.791511


In [151]:

history_test = EC.history.copy()


In [152]:

history_test.corr()


,result,pregame_rating,pregame_opp_rating
result,1.000000,0.214158,-0.214158
pregame_rating,0.214158,1.000000,0.402717
pregame_opp_rating,-0.214158,0.402717,1.000000


In [123]:

rating_period


,date,season,team_name,opp_name,is_home,stat,difference,result,rating_period,protag_idx,antag_idx,stat_idx
413116,2022-04-04,2022,Kansas,North Carolina,0,reb_diff,-15,0,2674.0,211,60,0
413117,2022-04-04,2022,Kansas,North Carolina,0,score_diff,3,1,2674.0,211,60,1
413118,2022-04-04,2022,North Carolina,Kansas,0,reb_diff,15,1,2674.0,60,211,0
413119,2022-04-04,2022,North Carolina,Kansas,0,score_diff,-3,0,2674.0,60,211,1


In [ ]:


self.rating_matrix[]

